In [17]:
import cv2, time
import mediapipe as mp
import tensorflow as tf
from tensorflow.python.keras.saving.save import load_model, load_weights
import numpy as np
import os
# import tensorflow_hub as hub
tf.test.is_built_with_cuda()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

ImportError: cannot import name 'load_weights' from 'tensorflow.python.keras.saving.save' (C:\Users\Natalia\anaconda3\envs\tfenv\lib\site-packages\tensorflow\python\keras\saving\save.py)

In [2]:
mp_face_detection = mp.solutions.face_detection

mp_drawing = mp.solutions.drawing_utils

In [3]:
os.getcwd()

'C:\\Users\\Natalia\\sortech'

In [16]:
labels = ['angry', 'disgusted', 'fear', 'happy', 'neutral', 'sad', 'surprised']
# model = tf.keras.models.load_model('D:\\Natalia\\sortech_emocje\\mobilenet-CK-EPOCHS_60test_acc_0.841.h5')
model = load_weights('C:\\Users\\Natalia\\sortech\\MODELE\\resnet50\\custom\\ck_jaffe_custom_resnet50_weights--EPOCHS_60test_acc_0.656.h5')
# model = load_model('C:\\Users\\Natalia\\sortech\\modele_all\\ck_jaffe_mobilenetV2_tuning-EPOCHS_60test_acc_0.675.h5', custom_objects={'KerasLayer':hub.KerasLayer})


NameError: name 'load_weights' is not defined

In [11]:
# cap = cv2.VideoCapture('filmy\GH011263.MP4')
# For webcam input:
cap = cv2.VideoCapture(0)

if (cap.isOpened() == False): 
  print("Unable to read camera feed")

cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)
# out = cv2.VideoWriter('output1.mp4', -1, 20.0, (640,480))
# out = cv2.VideoWriter('output.mp4', -1, 5.0, size)  #set 20 if video and 10 if webcam


In [12]:
with mp_face_detection.FaceDetection(
    model_selection=1, min_detection_confidence=0.75) as face_detection: 
    while cap.isOpened():
        success, image = cap.read()
        if success:
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)

            # Draw the face detection annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.detections:
                for detection in results.detections:
                    location_data = detection.location_data

                    if location_data.format == location_data.RELATIVE_BOUNDING_BOX:
                        bb = location_data.relative_bounding_box

                        ih, iw, ic = image.shape

                        box = int(bb.xmin*iw) , int(bb.ymin*ih), int(bb.width*iw), int(bb.height*ih)

                        ymin = int(bb.ymin*ih)
                        ymax = int(bb.height*ih) +ymin
                        xmin =  int(bb.xmin*iw)
                        xmax = int(bb.width*iw) +xmin
                        start = xmin, ymin
                        end = xmax, ymax


                        cv2.rectangle(image, start, end,(255, 0, 255) , 3 )   
                        cv2.rectangle(image, (xmin, ymin), (xmin +180, ymin+20), (255, 0, 255), -1)
                        cropped = image[ymin:ymax , xmin: xmax]
                        if cropped.shape[0]*cropped.shape[1] > 0:
                            face = cv2.resize(cropped, (224,224)) 
                            face = face/255.0
                        else:
                            continue



            # predictions = model.predict(np.array([face.reshape(224,224,3)])).argmax()
            predictions = model.predict(np.array([face.reshape(224,224,3)]))
            state = str(labels[predictions.argmax()])
            percent = str(format(predictions.max(), '.2f'))
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image,state+" "+percent, (xmin+10,ymin+15) , font, 0.6, (0,0,0), 2, cv2.LINE_AA)
            print(predictions)
            print(state, predictions.max())
        else:
            continue
            # out.write(image)
        cv2.imshow('MediaPipe Face Detection', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
# out.release()
cv2.destroyAllWindows()

[[5.1228769e-02 9.2972475e-01 1.0055823e-03 8.5354745e-03 1.7104740e-03
  7.7409963e-03 5.3899857e-05]]
disgusted 0.92972475
[[5.1528037e-02 9.2912638e-01 1.0272844e-03 8.6482931e-03 1.7499012e-03
  7.8648739e-03 5.5320892e-05]]
disgusted 0.9291264
[[5.2110389e-02 9.2821670e-01 1.0469819e-03 8.7503567e-03 1.7929209e-03
  8.0260197e-03 5.6593643e-05]]
disgusted 0.9282167
[[5.2500445e-02 9.2762381e-01 1.0629849e-03 8.7612122e-03 1.8295537e-03
  8.1647243e-03 5.7183424e-05]]
disgusted 0.9276238
[[5.3187262e-02 9.2660308e-01 1.0865948e-03 8.8500883e-03 1.8753299e-03
  8.3389822e-03 5.8702033e-05]]
disgusted 0.9266031
[[5.3062584e-02 9.2678601e-01 1.0848341e-03 8.8260379e-03 1.8734162e-03
  8.3088009e-03 5.8308007e-05]]
disgusted 0.926786
[[5.2950624e-02 9.2692435e-01 1.0798331e-03 8.8179195e-03 1.8679503e-03
  8.3006509e-03 5.8624635e-05]]
disgusted 0.92692435
[[5.3275786e-02 9.2636585e-01 1.0974361e-03 8.9080678e-03 1.8971997e-03
  8.3960099e-03 5.9666989e-05]]
disgusted 0.92636585
[[5.23

[[5.2654915e-02 9.2697382e-01 1.0948769e-03 9.1232648e-03 1.8587230e-03
  8.2343938e-03 5.9950864e-05]]
disgusted 0.9269738
[[5.2910615e-02 9.2666286e-01 1.0989285e-03 9.1317231e-03 1.8635369e-03
  8.2722623e-03 6.0051414e-05]]
disgusted 0.92666286
[[5.2904855e-02 9.2667842e-01 1.0960132e-03 9.1516497e-03 1.8550109e-03
  8.2541648e-03 5.9883136e-05]]
disgusted 0.9266784
[[5.3150088e-02 9.2628986e-01 1.1087352e-03 9.1581391e-03 1.8847205e-03
  8.3480282e-03 6.0493996e-05]]
disgusted 0.92628986
[[5.2971661e-02 9.2663008e-01 1.0953245e-03 9.1147823e-03 1.8599342e-03
  8.2683694e-03 5.9877217e-05]]
disgusted 0.9266301
[[5.2971661e-02 9.2663008e-01 1.0953245e-03 9.1147823e-03 1.8599342e-03
  8.2683694e-03 5.9877217e-05]]
disgusted 0.9266301
[[5.3052835e-02 9.2647332e-01 1.1001916e-03 9.1513088e-03 1.8674250e-03
  8.2945628e-03 6.0340484e-05]]
disgusted 0.9264733
[[5.3048447e-02 9.2655259e-01 1.0982933e-03 9.0805748e-03 1.8673978e-03
  8.2925986e-03 6.0091526e-05]]
disgusted 0.9265526
[[5.31

[[5.2306417e-02 9.2765450e-01 1.0685086e-03 9.0543404e-03 1.7860113e-03
  8.0718361e-03 5.8376681e-05]]
disgusted 0.9276545
[[5.2339248e-02 9.2759806e-01 1.0685317e-03 9.0594646e-03 1.7967694e-03
  8.0796499e-03 5.8281403e-05]]
disgusted 0.92759806
[[5.2605428e-02 9.2724949e-01 1.0751045e-03 9.0503450e-03 1.8119531e-03
  8.1487605e-03 5.8874724e-05]]
disgusted 0.9272495
[[5.2715346e-02 9.2705601e-01 1.0800355e-03 9.0911342e-03 1.8230422e-03
  8.1751291e-03 5.9391466e-05]]
disgusted 0.927056
[[5.2542578e-02 9.2742056e-01 1.0660546e-03 9.0223756e-03 1.8020382e-03
  8.0880020e-03 5.8308666e-05]]
disgusted 0.92742056
[[5.2715171e-02 9.2704147e-01 1.0812753e-03 9.1097197e-03 1.8231006e-03
  8.1697125e-03 5.9574217e-05]]
disgusted 0.9270415
[[5.2223455e-02 9.2772603e-01 1.0723406e-03 9.0315286e-03 1.8032163e-03
  8.0853626e-03 5.8148471e-05]]
disgusted 0.92772603
